In [ ]:
from google.colab import drive
import os

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Verify the drive is mounted before proceeding
if not os.path.exists('/content/drive/MyDrive'):
    raise Exception("Google Drive not mounted. Please check permissions and try again.")
else:
    print("Drive mounted successfully.")

# This is the FINAL destination for the installed software
install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"

# Create the directory if it doesn't already exist
os.makedirs(install_dir, exist_ok=True)
print(f"Installation directory ensured at: {install_dir}")

Mounting Google Drive...
Mounted at /content/drive
Drive mounted successfully.
Installation directory ensured at: /content/drive/MyDrive/Colab_Software/GROMACS


## GROMACS Installation

This cell handles the installation of GROMACS to your Google Drive. It checks for an existing installation and, if not found, downloads, compiles, and installs the specified GROMACS version.

In [ ]:
!rm -rf "/content/drive/MyDrive/Colab_Software/GROMACS"

In [ ]:
import os
import subprocess

# --- Configuration ---
# I've updated the version to 2025.3 to match your .tpr files
install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
gromacs_version = "2025.3"
gmx_binary = os.path.join(install_dir, "bin", "gmx")

# --- 1. Check for Existing Installation ---
print("--- Checking for existing GROMACS installation... ---")

# 1a. CRITICAL FIX: Ensure execute permission is set (for Drive files)
if os.path.exists(gmx_binary):
    !chmod +x {gmx_binary}

# 1b. Check if the installed binary runs and prints the correct version
try:
    result = subprocess.run(
        f'{gmx_binary} -version',
        shell=True,
        capture_output=True,
        text=True,
        check=True
    )
    if f"GROMACS version {gromacs_version}" in result.stdout:
        print(f"✅ GROMACS {gromacs_version} is already installed and verified. Skipping compilation.")
        gromacs_install_needed = False
    else:
        print(f"GROMACS found but version is incorrect. Reinstalling.")
        gromacs_install_needed = True

except (subprocess.CalledProcessError, FileNotFoundError):
    print("GROMACS not found or failed to run. Starting fresh installation.")
    gromacs_install_needed = True


if gromacs_install_needed:
    # --- Proceed with Installation ---
    print("--- Starting GROMACS Installation ---")

    build_root = "/content/gromacs_build"
    gromacs_tarball = f"gromacs-{gromacs_version}.tar.gz"
    gromacs_src_dir = f"gromacs-{gromacs_version}"
    build_dir = os.path.join(build_root, gromacs_src_dir, "build")

    # Clean up the build directory before starting
    if os.path.exists(build_root):
        !rm -rf {build_root}

    os.makedirs(build_root, exist_ok=True)
    os.chdir(build_root)
    print(f"Changed to temporary build directory: {os.getcwd()}")

    # Download
    print(f"Downloading {gromacs_tarball}...")
    # Updated to handle potential mirror changes, will try multiple sources
    try:
        !wget -nc http://ftp.gromacs.org/pub/gromacs/{gromacs_tarball}
    except Exception as e:
        print(f"Primary download failed, trying alternative. Error: {e}")
        !wget -nc https://manual.gromacs.org/documentation/{gromacs_version}/download/{gromacs_tarball}


    # Extract
    print(f"Extracting {gromacs_tarball}...")
    !tar xzf {gromacs_tarball}

    # Configure (CMake)
    os.makedirs(build_dir, exist_ok=True)
    os.chdir(build_dir)
    print(f"Changed to build directory: {os.getcwd()}")
    print("Configuring GROMACS with CMake...")
    !cmake .. -DGMX_GPU=CUDA -DCMAKE_INSTALL_PREFIX={install_dir} -DGMX_BUILD_OWN_FFTW=ON

    # Compile (Make)
    print("Compiling GROMACS... (This will take 30-60 minutes)")
    !make -j $(nproc)

    # Install (Make Install)
    print("Installing GROMACS to Google Drive...")
    !make install

    # Final permissions check
    print(f"Ensuring execute permissions for {gmx_binary}")
    !chmod +x {gmx_binary}

    print(f"--- GROMACS {gromacs_version} installation complete! Files are in {install_dir} ---")

    # Clean up build files from local storage
    os.chdir("/content")
    !rm -rf {build_root}
    print(f"Cleaned up temporary build directory: {build_root}")

--- Checking for existing GROMACS installation... ---
GROMACS not found or failed to run. Starting fresh installation.
--- Starting GROMACS Installation ---
Changed to temporary build directory: /content/gromacs_build
--2025-10-19 17:56:32--  http://ftp.gromacs.org/pub/gromacs/gromacs-2025.3.tar.gz
Resolving ftp.gromacs.org (ftp.gromacs.org)... 130.237.11.165, 2001:6b0:1:1191:216:3eff:fec7:6e30
Connecting to ftp.gromacs.org (ftp.gromacs.org)|130.237.11.165|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ftp.gromacs.org/pub/gromacs/gromacs-2025.3.tar.gz [following]
--2025-10-19 17:56:33--  https://ftp.gromacs.org/pub/gromacs/gromacs-2025.3.tar.gz
Connecting to ftp.gromacs.org (ftp.gromacs.org)|130.237.11.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44407119 (42M) [application/x-gzip]
Saving to: ‘gromacs-2025.3.tar.gz’

gromacs-2025.3.tar. 100%[===================>]  42.35M  21.5MB/s    in 2.0s    

2025-1

## Setting up GROMACS Environment Variables

This cell creates a shell script to set up the necessary environment variables for the GROMACS installation. This allows you to easily access the GROMACS binaries and libraries from any terminal session after sourcing the script.

In [ ]:
import os

install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
env_vars_file = os.path.join(install_dir, "gromacs_env.sh")

# Create the shell script if it doesn't exist or is empty
if not os.path.exists(env_vars_file) or os.path.getsize(env_vars_file) == 0:
    print(f"Creating environment variables script at: {env_vars_file}")
    with open(env_vars_file, "w") as f:
        f.write(f'# Environment script for GROMACS installed to {install_dir}\n')
        f.write(f'export GMXBIN="{install_dir}/bin"\n')
        f.write(f'export GMXLIB="{install_dir}/share/gromacs/top"\n')
        f.write(f'export PATH="{install_dir}/bin:$PATH"\n')
        f.write(f'export LD_LIBRARY_PATH="{install_dir}/lib:$LD_LIBRARY_PATH"\n')
        f.write(f'export PKG_CONFIG_PATH="{install_dir}/lib/pkgconfig:$PKG_CONFIG_PATH"\n')
else:
    print(f"Environment script already exists at: {env_vars_file}")

# --- Test it in the current session ---
print("\n--- Sourcing script and testing GROMACS version ---")
!source {env_vars_file} && chmod +x {install_dir}/bin/gmx && gmx -version

Creating environment variables script at: /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh

--- Sourcing script and testing GROMACS version ---
                         :-) GROMACS - gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content
Command line:
  gmx -version

GROMACS version:     2025.3
Precision:           mixed
Memory model:        64 bit
MPI library:         thread_mpi
OpenMP support:      enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:         CUDA
NBNxM GPU setup:     super-cluster 2x2x2 / cluster 8 (cluster-pair splitting on)
SIMD instructions:   AVX2_256
CPU FFT library:     fftw-3.3.10-sse2-avx-avx2-avx2_128
GPU FFT library:     cuFFT
Multi-GPU FFT:       none
RDTSCP usage:        enabled
TNG support:         enabled
Hwloc support:       disabled
Tracing support:     disabled
C compiler:          /usr/bin/cc GNU 11.4.0
C compiler flags:    -fe

## Installing condacolab

This cell installs `condacolab`, a package that enables the use of Conda environments within Google Colab. Running this cell will restart the Colab runtime.

In [ ]:
# --- Install gmxMMPBSA (Requires Conda) ---
print("\n--- Installing condacolab. This will restart the Colab runtime. ---")
print("After the restart, run the next cell (Step 5).")

!pip install -q condacolab
import condacolab
condacolab.install()


--- Installing condacolab. This will restart the Colab runtime. ---
After the restart, run the next cell (Step 5).
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


## Installing Analysis Tools (gmxMMPBSA and others)

This cell sets up a dedicated Conda environment and installs various analysis tools, including `gmx_MMPBSA`, AmberTools, and other Python packages useful for molecular dynamics analysis. It checks for an existing installation before proceeding.

In [ ]:
import os
import subprocess

conda_env_name = "gmxmmpbsa"

# --- 1. Check for Existing Conda Environment ---
print(f"--- Checking for existing Conda environment '{conda_env_name}'... ---")
try:
    # Check if the environment exists and gmx_MMPBSA is installed
    result = subprocess.run(
        f'conda run -n {conda_env_name} gmx_MMPBSA -v',
        shell=True,
        capture_output=True,
        text=True,
        check=True
    )
    if "gmx_MMPBSA" in result.stdout:
        print(f"✅ Conda environment '{conda_env_name}' and gmx_MMPBSA are verified. Skipping installation.")
        conda_install_needed = False
    else:
        print(f"Conda environment '{conda_env_name}' found, but gmx_MMPBSA verification failed. Reinstalling.")
        conda_install_needed = True
except (subprocess.CalledProcessError, FileNotFoundError):
    print(f"Conda environment '{conda_env_name}' not found or failed to run. Starting fresh installation.")
    conda_install_needed = True


if conda_install_needed:
    print("--- Starting Analysis Tools Installation ---")

    print("--- 5.1: Installing System Dependencies (DFT libs, ImageMagick) ---")
    !apt-get update -y
    !apt-get install -y libxc-dev liblapack-dev libblas-dev imagemagick

    print(f"\n--- 5.2: Creating '{conda_env_name}' Conda Environment ---")
    !conda create -n {conda_env_name} python=3.9 -y

    print("\n--- 5.3: Installing AmberTools & Compilers ---")
    !conda run -n {conda_env_name} conda install -c conda-forge ambertools compilers mpi4py -y

    print("\n--- 5.4: Installing All Python Analysis Tools ---")
    !conda run -n {conda_env_name} pip install gmx_MMPBSA acpype MDAnalysis prolif pandas matplotlib seaborn

    print(f"\n--- ✅ Installation of Analysis Tools Finished ---")

print("\n--- Verifying gmxMMPBSA installation: ---")
!conda run -n {conda_env_name} gmx_MMPBSA -v

--- Checking for existing Conda environment 'gmxmmpbsa'... ---
Conda environment 'gmxmmpbsa' not found or failed to run. Starting fresh installation.
--- Starting Analysis Tools Installation ---
--- 5.1: Installing System Dependencies (DFT libs, ImageMagick) ---
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRel

In [ ]:
import os

install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
env_vars_file = os.path.join(install_dir, "gromacs_env.sh")

!source {env_vars_file} && gmx --version
!nvidia-smi

                         :-) GROMACS - gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content
Command line:
  gmx --version

GROMACS version:     2025.3
Precision:           mixed
Memory model:        64 bit
MPI library:         thread_mpi
OpenMP support:      enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:         CUDA
NBNxM GPU setup:     super-cluster 2x2x2 / cluster 8 (cluster-pair splitting on)
SIMD instructions:   AVX2_256
CPU FFT library:     fftw-3.3.10-sse2-avx-avx2-avx2_128
GPU FFT library:     cuFFT
Multi-GPU FFT:       none
RDTSCP usage:        enabled
TNG support:         enabled
Hwloc support:       disabled
Tracing support:     disabled
C compiler:          /usr/bin/cc GNU 11.4.0
C compiler flags:    -fexcess-precision=fast -funroll-all-loops -mavx2 -mfma -Wno-missing-field-initializers -O3 -DNDEBUG
C++ compiler:        /usr/bin/c++ GNU 11.4.0
C++ compiler

In [ ]:
print("MD run has finished.")
print("Shutting down Colab runtime to save points...")

from google.colab import runtime
runtime.unassign()